In [1]:
import os.path
import requests
import pandas as pd
# from download_file import download_file
from utils import save2file, read_cache, set_cache

In [2]:
import os
import uuid
import requests
from document_convert import ocr_pdf, read_doc, read_docx, read_wps

headers = {
    'Accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Host': 'fgsjk.sxpc.gov.cn:8443',
    'Referer': 'http://fgsjk.sxpc.gov.cn:8443/document-detail?id=8650cf7a6009ae3b0b457f7a4412058c',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
}
#  url = f'http://fgsjk.sxpc.gov.cn:8443/prod-api/gx-attachment/attachment/obtain/download/{file_id}'

def download_file(file_id: str) -> str:
    params = {
        'module': 'gx-regulationdb-intranet',
        'id': file_id,
    }

    response = requests.get(
        'http://fgsjk.sxpc.gov.cn:8443/prod-api/gx-attachment/attachment/obtain/download',
        params=params,
        headers=headers,
        verify=False,
        timeout=10,
    )
    if response.status_code != 200:
        raise Exception(f'下载失败: {response.text}')
        # 获取文件名和后缀
    filename_final = response.headers.get('Content-Disposition')
    if filename_final:
        filename_final = filename_final.split('.')[-1].lower()
    else:
        filename_final = 'doc'

    file_path = f'/home/bld/data/data3/peng/Bridge/山西/tmp/{str(uuid.uuid4())}.{filename_final}'

    with open(file_path, 'wb') as f:
        f.write(response.content)
    
    if filename_final in ('doc', 'wps', 'docx','pdf'): # 如果是 doc, wps, docx, pdf 文件
        # 如果是 doc文件，读取内容
        if filename_final == 'doc':
            text = read_doc(file_path)
        # 如果是 docx文件，读取内容
        elif filename_final == 'docx':
            text = read_docx(file_path)
        # 如果是 wps文件，读取内容
        elif filename_final == 'wps':
            text = read_wps(file_path)
        # 如果是 pdf文件，读取内容
        elif filename_final == 'pdf':
            text = ocr_pdf(file_path)
        # os.remove(file_path) # 删除原文件
        return text

[2024/09/26 07:08:54] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=2, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/bld/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/bld/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25

In [3]:
SAVE_PATH = '/home/bld/data/data3/peng/Bridge/山西/data/detail.txt'

headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Pragma': 'no-cache',
    'Referer': 'http://fgsjk.sxpc.gov.cn:8443/prod-api/extranet/regulation/search',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0',
}

In [4]:
def get_base_info(page_id: str) -> dict:
    url = f'http://fgsjk.sxpc.gov.cn:8443/prod-api/extranet/regulation/getById/{page_id}'
    print(f'开始爬取：{f"http://fgsjk.sxpc.gov.cn:8443/document-detail?id={page_id}&themeConfigId=undefined"}')
    response = requests.get(
        url,
        headers=headers,
        verify=False,
        timeout=10,
    )
    resp_json = response.json() 
    if resp_json.get('code', '') != 200:
        if 'Unmatched closing' in resp_json.get('msg', '') or 'Unclosed group near index' in resp_json.get('msg', ''):
            return {}
        raise Exception(f'获取详情失败: {resp_json}')

    file_id = None
    preferred_type = 'doc'
    fallback_type = 'pdf'

    # 优先选择 DOC 文件
    for file in resp_json['data']['regulationFiles']:
        if file['attType'] == preferred_type:
            file_id = file['attId']
            break

    # 如果没有找到 DOC 文件，再选择 PDF 文件
    if not file_id:
        for file in resp_json['data']['regulationFiles']:
            if file['attType'] == fallback_type:
                file_id = file['attId']
                break
    print(resp_json['data'])
    # return {
    #     '文件标题': resp_json['data']['title'],
    #     '法律效力位阶': resp_json['data']['filetypeVo']['name'],
    #     '制定形式': resp_json['data']['formulateMode'],
    #     '时效性': resp_json['data']['timeliness'],
    #     '制定机关名称': resp_json['data']['officeVo']['name'] if resp_json['data']['officeVo'] else None,
    #     '发布日期': resp_json['data']['publishDate'],
    #     '施行日期': resp_json['data']['expiryDate'],
    #     '发文字号': resp_json['data']['releaseNum'],
    #     '主题分类': resp_json['data']['tagNames'],
    #     '通过日期': resp_json['data'].get('passDate', None),
    #     '页面地址': page_id,
    #     '文件地址': file_id
    # }
    return {
    '文件标题': resp_json['data'].get('title', None),
    '法律效力位阶': resp_json['data'].get('filetypeVo', {}).get('name', None),
    '制定形式': resp_json['data'].get('formulateMode', None),
    '时效性': resp_json['data'].get('timeliness', None),
    '制定机关名称': resp_json['data']['officeVo']['name'] if resp_json['data']['officeVo'] else None,
    '发布日期': resp_json['data'].get('publishDate', None),
    '施行日期': resp_json['data'].get('expiryDate', None),
    '发文字号': resp_json['data'].get('releaseNum', None),
    '主题分类': resp_json['data'].get('tagNames', None),
    '通过日期': resp_json['data'].get('passDate', None),
    '页面地址': page_id,
    '文件地址': file_id
}

In [5]:
page_id = '2feeba77b4bcf105d1bda9e075c53a4b'
base_info = get_base_info(page_id)
base_info

开始爬取：http://fgsjk.sxpc.gov.cn:8443/document-detail?id=2feeba77b4bcf105d1bda9e075c53a4b&themeConfigId=undefined
{'id': '2feeba77b4bcf105d1bda9e075c53a4b', 'filetypeVo': {'id': 'b1b766d904843678680fc64b6efc00b3', 'type': None, 'name': '县（市、区）政府及办公室规范性文件', 'value': None, 'parentId': '0cea6d39e3a6fd74f5196b84aa7cb989', 'parentIds': '0,9f22b2c0663d88043fdfa6a73cceef32,0cea6d39e3a6fd74f5196b84aa7cb989,', 'orderNum': 89, 'status': None, 'regionName': None, 'regionCode': None, 'childs': None}, 'officeVo': {'id': '12001', 'name': '永济市人民政府办公室（市政府外事办公室、市政府台港澳事务办公室）', 'orderNum': 12001, 'regionCode': '140881', 'regionName': '永济市', 'status': '0', 'childs': None, 'show': False}, 'businessType': 'normativeDocuments', 'title': '永济市人民政府办公室关于印发《永济市农村自建房建设管理办法（试行）》的通知', 'subTitle': None, 'releaseNum': None, 'publishDate': '2023-05-22', 'expiryDate': '2023-05-22', 'timeliness': '现行有效', 'formulateMode': '制定', 'regionCode': '140881', 'regionName': '永济市', 'regulationFiles': [{'id': '6fb574599c6e26953ced168d0

{'文件标题': '永济市人民政府办公室关于印发《永济市农村自建房建设管理办法（试行）》的通知',
 '法律效力位阶': '县（市、区）政府及办公室规范性文件',
 '制定形式': '制定',
 '时效性': '现行有效',
 '制定机关名称': '永济市人民政府办公室（市政府外事办公室、市政府台港澳事务办公室）',
 '发布日期': '2023-05-22',
 '施行日期': '2023-05-22',
 '发文字号': None,
 '主题分类': '',
 '通过日期': None,
 '页面地址': '2feeba77b4bcf105d1bda9e075c53a4b',
 '文件地址': '4fa43c61a9f48413695ce042a25f0a38'}

In [11]:
csv = pd.read_csv('/home/bld/data/data3/peng/Bridge/山西/data/list.csv')
csv

,id,title,file_type,timeliness,file_id,page
0,c853d49807f03264b7792ec8f997549a,闻喜县人民政府关于更新城区基准地价的通知,县（市、区）政府及办公室规范性文件,现行有效,aa647c0580a6709a04977346dd41d335,1.0
1,8650cf7a6009ae3b0b457f7a4412058c,翼城县人民政府关于坚决制止耕地“非农化”防止耕地”非粮化“的通告.,县（市、区）政府及办公室规范性文件,现行有效,b44f1a503cbdaf708df4041b00eb5ef0,1.0
2,07c15dad1de32a42702cf333ed8421e3,运城市电梯安全管理条例,设区的市地方性法规,现行有效,057516d7c7512d4d78c4bc3e7e93a8ea,1.0
3,4e845dbd6e457f74e9f9ebf4eb1f5a93,晋中市退役军人事务局 晋中市财政局 关于印发《晋中市军队离休退休干部荣誉疗养办法》的通知,市政府部门行政规范性文件,现行有效,2765187cb1bea00c6dd50d8130ddfff7,1.0
4,162f7d155bf4b597e6ca2f3686944d87,临汾市沁河流域生态保护与修复条例,设区的市地方性法规,现行有效,2ca0326a82a1d3343041f9654a9d390b,1.0
...,...,...,...,...,...,...
4034,fc567bb6e02871c206198042948b243e,屯留县人大常委会关于批准屯留县人民政府使用财政预算资金支付我县城市改造项目房屋征收补偿款有关...,县（市、区）人大及其常委会规范性文件,NaN,a73c09315ecdc13952c13a7763a8f044,101.0
4035,fce85e00b38da858072ad1083b177ccd,灵石县人民政府办公室 关于印发灵石县企业应急周转保障资金 管理办法的通知,县（市、区）政府及办公室规范性文件,已修改,c590418083ed8169f7195ccf14c644e8,101.0
4036,fd34eaf21bb8e2c128dcabc8799b04a8,并教法〔2021〕1号关于公布《太原市教育局行政处罚裁量权适用参照标准》的通知,市政府部门行政规范性文件,NaN,52916d3a29010cbc5b3c9bb539b80ac5,101.0
4037,fd542c1f6c03d005603928f56ccd11a8,永和县人民政府办公室关于印发《永和县扶贫小额信贷风险补偿金管理办法》的通知,县（市、区）政府及办公室规范性文件,NaN,8ff6bad9de59da834a1dd47d9ce46bc8,101.0


In [16]:
# 查找存在空值的行
rows_with_nulls = csv[csv.isnull().any(axis=1)]
rows_with_nulls 

,id,title,file_type,timeliness,file_id,page
12,d29f50050c353b2fc3c03108142095d7,全省农村集体经济组织财务制度实施细则（试行）,NaN,NaN,NaN,NaN
13,c885bf6bfb8de40018b9fd6a580f1c44,省政府部门行政规范性文件,现行有效,4028dc78b83a58956f5b4fdc7fd57bf1,1,NaN
113,5bcb9e6d3d159a3b735d9d1b37a580cd,阳泉市人民政府办公室关于印发阳泉市道路货物运输源头科技治超管理服务办法的通知,市政府及办公室规范性文件,NaN,1708e22a7acc23512bcd0afc476f37fe,3.0
187,a7ec51cba17441ebb765cf951ceb1375,阳泉市人民政府办公室关于印发《阳泉市电动汽车充（换）电基础设施建设运营管理办法（试行）》的通知,市政府及办公室规范性文件,NaN,b31b3c226ca33b1a3ca3be16a512499f,5.0
240,020dd6e0d15d33d148d566bdcb9a91fc,山西省高级人民法院 山西省人民检察院 《关于建立常态化工作交流会商机制的意见（试行）》,省检察院规范性文件,NaN,92c7c8a307c569a348e54eeebefef05c,6.0
...,...,...,...,...,...,...
4033,facaf059f00fbc7345cc15ffa36552f4,平陆县人民代表大会常务委员会关于开展第八个五年法治宣传教育的决议,县（市、区）人大及其常委会规范性文件,NaN,5527f3e13efa7407f916c21053bcbe44,101.0
4034,fc567bb6e02871c206198042948b243e,屯留县人大常委会关于批准屯留县人民政府使用财政预算资金支付我县城市改造项目房屋征收补偿款有关...,县（市、区）人大及其常委会规范性文件,NaN,a73c09315ecdc13952c13a7763a8f044,101.0
4036,fd34eaf21bb8e2c128dcabc8799b04a8,并教法〔2021〕1号关于公布《太原市教育局行政处罚裁量权适用参照标准》的通知,市政府部门行政规范性文件,NaN,52916d3a29010cbc5b3c9bb539b80ac5,101.0
4037,fd542c1f6c03d005603928f56ccd11a8,永和县人民政府办公室关于印发《永和县扶贫小额信贷风险补偿金管理办法》的通知,县（市、区）政府及办公室规范性文件,NaN,8ff6bad9de59da834a1dd47d9ce46bc8,101.0


In [17]:
csv['id'].isnull().sum()

0

In [18]:
# 找出id为空的行
rows_with_nullsid = csv[csv['id'].isnull()]
rows_with_nullsid

,id,title,file_type,timeliness,file_id,page


In [19]:
csv['file_id'].isnull().sum()

11

In [20]:
# 找出file_id为空的行
rows_with_nullsfile_id = csv[csv['file_id'].isnull()]
rows_with_nullsfile_id

,id,title,file_type,timeliness,file_id,page
12,d29f50050c353b2fc3c03108142095d7,全省农村集体经济组织财务制度实施细则（试行）,NaN,NaN,NaN,NaN
780,cd275cc7bc70793c1c567d761cd1119b,晋城市中级人民法院 晋城市人民检察院 晋城市人力资源和社会保障局印发《关于建立工商保险协作机...,市检察院规范性文件,NaN,NaN,20.0
913,089510259a2d5eaf38de09b9feee514e,关于印发《山西省林草行政执法与刑事司法衔接工作实施办法（试行）》的通知,省检察院规范性文件,NaN,NaN,23.0
937,c84078265b2dc453067223c712540660,关于印发《关于在办理涉未成年人案件中全面开展家庭教育指导工作的实施意见（试行）》的通知,省检察院规范性文件,NaN,NaN,24.0
1228,810d98b199100e28b3c41fa470000c1d,关于加强寄递安全监管协作配合工作机制的意见（试行）,市检察院规范性文件,NaN,NaN,31.0
1931,de6daee29e3d45a9e08f86990070fe37,关于印发《关于服务保障黄河流域生态保护和高质量发展加强协作的意见》,省检察院规范性文件,NaN,NaN,49.0
1944,7926407a8cc35673e1cdafc037d67bc2,山西省高级人民法院 山西省人民检察院 山西省公安厅《关于防范和惩治虚假诉讼的若干规定》,省检察院规范性文件,NaN,NaN,49.0
2406,73611fafbf7dc07f59ed2eae57ed0cb4,关于印发《忻州市人民检察院 忻州市妇女联合会关于建立妇女儿童权益保护联动机制》的通知,市检察院规范性文件,NaN,NaN,61.0
2807,be1888e02f98f36b00603a37ac67b93e,关于印发《全省检察机关支持和服务企业家创新创业营造良好法治环境的意见》的通知,省检察院规范性文件,NaN,NaN,71.0
2923,c9163c6495498edece6f6ecf9dff52c4,关于印发《山西省人民检察院关于充分发挥检察职能服务保障优化营商环境的实施意见》的通知,省检察院规范性文件,NaN,NaN,74.0


In [30]:
def spider_detail(row) -> dict:
    page_info = get_base_info(row['id'])
    if not page_info:
        # 这种就是页面报错的！！！
        return {
            'url': f'http://fgsjk.sxpc.gov.cn:8443/document-detail?id={row["id"]}&searchValue=',
            '标题': row['title'],
        }
    file_id = page_info.pop('文件地址', None)
    if file_id:
        try:
            doc_text = download_file(file_id)
        except Exception as e:
            print(e)
            doc_text = ''
    else:
        doc_text = ''
    _result = {
        'url': f'http://fgsjk.sxpc.gov.cn:8443/document-detail?id={row["id"]}&searchValue=',
        '标题': row['title'],
        **page_info, 
        '文件内容': doc_text.strip() if doc_text else '',
        'file_url': f'http://fgsjk.sxpc.gov.cn:8443/prod-api/gx-attachment/attachment/obtain/download?module=gx-regulationdb-intranet&id={file_id}' if file_id else None,
    }
    return _result

In [6]:
def spider_detail(row) -> dict:
    page_info = get_base_info(row['id'])
    if not page_info:
        # 这种就是页面报错的！！！
        return {
            'url': f'http://fgsjk.sxpc.gov.cn:8443/document-detail?id={row["id"]}&searchValue=',
            '标题': row['title'],
        }
    file_id = page_info.pop('文件地址', None)
    print(f'file_id: {file_id}')
    if file_id:
        try:
            doc_text = download_file(file_id)
        except Exception as e:
            print(e)
            doc_text = ''
    else:
        doc_text = ''
    _result = {
        'url': f'http://fgsjk.sxpc.gov.cn:8443/document-detail?id={row["id"]}&searchValue=',
        '标题': row['title'],
        **page_info, 
        '文件内容': doc_text.strip() if doc_text else '',
        'file_url': f'http://fgsjk.sxpc.gov.cn:8443/prod-api/gx-attachment/attachment/obtain/preview-word2pdf/{file_id}?module=gx-regulationdb-intranet' if file_id else None,
    }
    return _result

In [8]:
# c853d49807f03264b7792ec8f997549a,闻喜县人民政府关于更新城区基准地价的通知,县（市、区）政府及办公室规范性文件,现行有效,aa647c0580a6709a04977346dd41d335,1
spider_detail({'id':'2feeba77b4bcf105d1bda9e075c53a4b', 'title':'22222'})

开始爬取：http://fgsjk.sxpc.gov.cn:8443/document-detail?id=2feeba77b4bcf105d1bda9e075c53a4b&themeConfigId=undefined
{'id': '2feeba77b4bcf105d1bda9e075c53a4b', 'filetypeVo': {'id': 'b1b766d904843678680fc64b6efc00b3', 'type': None, 'name': '县（市、区）政府及办公室规范性文件', 'value': None, 'parentId': '0cea6d39e3a6fd74f5196b84aa7cb989', 'parentIds': '0,9f22b2c0663d88043fdfa6a73cceef32,0cea6d39e3a6fd74f5196b84aa7cb989,', 'orderNum': 89, 'status': None, 'regionName': None, 'regionCode': None, 'childs': None}, 'officeVo': {'id': '12001', 'name': '永济市人民政府办公室（市政府外事办公室、市政府台港澳事务办公室）', 'orderNum': 12001, 'regionCode': '140881', 'regionName': '永济市', 'status': '0', 'childs': None, 'show': False}, 'businessType': 'normativeDocuments', 'title': '永济市人民政府办公室关于印发《永济市农村自建房建设管理办法（试行）》的通知', 'subTitle': None, 'releaseNum': None, 'publishDate': '2023-05-22', 'expiryDate': '2023-05-22', 'timeliness': '现行有效', 'formulateMode': '制定', 'regionCode': '140881', 'regionName': '永济市', 'regulationFiles': [{'id': '6fb574599c6e26953ced168d0

{'url': 'http://fgsjk.sxpc.gov.cn:8443/document-detail?id=2feeba77b4bcf105d1bda9e075c53a4b&searchValue=',
 '标题': '22222',
 '文件标题': '永济市人民政府办公室关于印发《永济市农村自建房建设管理办法（试行）》的通知',
 '法律效力位阶': '县（市、区）政府及办公室规范性文件',
 '制定形式': '制定',
 '时效性': '现行有效',
 '制定机关名称': '永济市人民政府办公室（市政府外事办公室、市政府台港澳事务办公室）',
 '发布日期': '2023-05-22',
 '施行日期': '2023-05-22',
 '发文字号': None,
 '主题分类': '',
 '通过日期': None,
 '页面地址': '2feeba77b4bcf105d1bda9e075c53a4b',
 '文件内容': '永济市人民政府办公室\n\n关于印发《永济市农村自建房建设管理办法\n\n(试行)》的通知\n\n各镇（街道），市直各有关单位，永济经济技术开发区管委会：\n\n《永济市农村自建房建设管理办法(试行)》已经市人民政府同意，现印发给你们，请认真贯彻执行。\n\n永济市人民政府办公室\n\n                                2023年5月22日\n\n（此件公开发布）\n\n永济市农村自建房建设管理办法(试行)\n\n第一章  总  则 \n\n第一条 为加强农村自建房建设管理，引导农民住宅建设合理、节约集约利用土地，促进农民集中居住，推进乡村振兴战略实施，根据山西省人民政府办公厅印发的“四办法一标准”，立足我市实际，制定本管理办法。\n\n    第二条 本管理办法适用于全市城镇开发边界以外农村集体建设用地上的新建、改建、扩建、翻建农村住房(以下统称“自建房”)的规划建设管理。\n\n第三条 本管理办法中下列用语的含义是：\n\n    （一）农村村民，是指具有农村集体经济组织成员资格的成员。\n\n    （二）农村宅基地，是指村民用于建造住宅及其附属设施的集体建设用地，包括住房、附属用房和庭院等用地。\n\n（三）农村自建低层房是由具有农村集体经济组织成员资格的人员，以户为单位申请在宅基地上建造的建

In [35]:
file = download_file('f5f2eb8b1a15671847e8d685816605d7')
file

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n\nTYBG―2023―4003\n并公积金〔2023〕13号\n\n关于印发《太原市住房公积金管理中心\n\n住房公积金行政执法管理办法》的通知\n\n铁路分中心，各分理处、科室：\n\n《太原市住房公积金管理中心住房公积金行政执法管理办法》已经太原住房公积金管理委员会五届一次会议审议通过，现予印发，自2023年 6月11日起实施，请遵照执行。\n\n太原市住房公积金管理中心    \n2023年5月11日  \n\n太原市住房公积金管理中心\n\n住房公积金行政执法管理办法\n\n第一章  总则\n\n第一条 为加强住房公积金行政执法管理，规范行政执法行为，维护职工的合法权益，根据《中华人民共和国行政处罚法》、《中华人民共和国行政强制法》、《中华人民共和国行政复议法》、《中华人民共和国行政诉讼法》、《住房公积金管理条例》（以下简称《条例》）等法律、法规的规定，结合本市实际，制定本办法。\n\n第二条 本办法所称住房公积金行政执法，是指太原市住房公积金管理中心(以下简称“公积金中心”)依法对违反住房公积金管理规定的单位进行调查取证、作出行政处理决定、实施行政处罚和申请人民法院强制执行的行为。\n\n第三条 本办法所称的行政处罚特指单位不办理住房公积金缴存登记或者不为本单位职工办理住房公积金账户设立手续的，由公积金中心责令限期办理；逾期不办理的，处1万元以上5万元以下的罚款。\n\n本办法所称的行政处理特指单位逾期不缴或少缴住房公积金的，由公积金中心责令限期缴存；逾期仍不缴存的，可以申请人民法院强制执行。\n第四条 本办法适用于本市行政区域内及公积金中心管辖的铁路系统通过投诉渠道进行的住房公积金行政执法活动。\n\n第五条 公积金中心实施行政执法必须以事实为根据，以法律为准绳，遵循公正、公开原则，坚持处罚与教育相结合，以教育整改为主、行政处罚为辅的原则。\n\n第六条 公积金中心应当依法建立健全和全面推行行政执法公示制度、执法全过程记录制度、重大执法决定法制审核制度。\n\n第二章  行政执法机构和人员\n\n第七条 公积金中心按照《条例》的规